In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_paths=[
    "/kaggle/input/ico-coffee-dataset-worldwide/domestic-consumption.csv",
    "/kaggle/input/ico-coffee-dataset-worldwide/exports-calendar-year.csv",
    "/kaggle/input/ico-coffee-dataset-worldwide/exports-crop-year.csv",
    "/kaggle/input/ico-coffee-dataset-worldwide/gross-opening-stocks.csv",
    "/kaggle/input/ico-coffee-dataset-worldwide/total-production.csv"
]

In [ ]:
dfs=[pd.read_csv(df_path) for df_path in df_paths]

In [ ]:
#making mean value of every df columns and attaching it to country

def get_means(df):
    df=df.copy()
    countries=df[df.columns[0]]
    mean=df.mean(axis=1)
    df=pd.concat([countries,mean],axis=1)
    df.columns=['country',countries.name]
    return df


In [ ]:
def make_df(dfs):
    
    # Process all DataFrames
    processed_dfs = []
    
    for df in dfs:
        processed_dfs.append(get_means(df))
        
    # Merge DataFrames
    df = processed_dfs[0]
    
    for i in range(1, len(processed_dfs)):
        df = df.merge(processed_dfs[i], on='country')
    
    return df

In [ ]:
data=make_df(dfs)
data

In [ ]:
def preprocess_inputs(df):
    df=df.copy()
    
    df=df.drop('country',axis=1)
    
    y=df['total_production']
    X=df.drop('total_production',axis=1)
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,shuffle=True,random_state=1)
    
    scaler=StandardScaler()
    scaler.fit(X_train)
    X_train=pd.DataFrame(scaler.transform(X_train),index=X_train.index,columns=X_train.columns)
    X_test=pd.DataFrame(scaler.transform(X_test),index=X_test.index,columns=X_test.columns)
    
    return X_train,X_test,y_train,y_test

In [ ]:
X_train,X_test,y_train,y_test=preprocess_inputs(data)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
model=RandomForestRegressor()
model.fit(X_train,y_train)
print("Model Trained")

In [ ]:
y_pred=model.predict(X_test)

rmse=np.sqrt(np.mean((y_test-y_pred)**2))
print("RMSE:{:.2f}".format(rmse))

r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
print("R^2: {:.5f}".format(r2))